In [25]:
import torch
import pandas as pd
from torchvision import transforms, datasets
from PIL import Image, ImageStat

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, DataQualityPreset, TargetDriftPreset
from evidently.test_suite import TestSuite
from evidently.tests import TestNumberOfMissingValues

In [2]:
def brightness( im_file ):
   im = Image.open(im_file).convert('L')
   stat = ImageStat.Stat(im)
   return stat.mean[0]

In [13]:

def get_data(path: str) -> list:
    """ Funtion to load the folders with the imgs to predict
    """
    test = pd.read_csv(path)
    paths = test["Path"].values
    test_labels = test["ClassId"].values

    transform = transforms.Compose(
        [
        transforms.Resize([32, 32]),
        transforms.ToTensor(),
        ]
    )

    test_imgs_means = []
    test_imgs_std = []

    for img_path in paths:
        fullpath = "data/raw/German/" + img_path
        img = Image.open(fullpath)
        mean = ImageStat.Stat(img).mean
        std = ImageStat.Stat(img).stddev
        test_imgs_means.append(mean)
        test_imgs_std.append(std)
        img.close()

    return test_imgs_means, test_imgs_std, test_labels

In [23]:
test_imgs_means_train, test_imgs_std_train, labels_train = get_data("data/raw/German/Train.csv")

image_data_train = pd.DataFrame(columns=["brightness_R", "brightness_G", "brightness_B", "contrast_R", "contrast_G", "contrast_B"])
image_data_train[["brightness_R", "brightness_G", "brightness_B"]] = test_imgs_means_train
image_data_train[[ "contrast_R", "contrast_G", "contrast_B"]] = test_imgs_std_train
image_data_train["labels"] = labels_train

In [24]:
test_imgs_means_test, test_imgs_std_test, labels_test = get_data("data/raw/German/Test.csv")

image_data_test = pd.DataFrame(columns=["brightness_R", "brightness_G", "brightness_B", "contrast_R", "contrast_G", "contrast_B"])
image_data_test[["brightness_R", "brightness_G", "brightness_B"]] = test_imgs_means_test
image_data_test[[ "contrast_R", "contrast_G", "contrast_B"]] = test_imgs_std_test
image_data_test["labels"] = labels_test

In [27]:
report = Report(metrics=[DataDriftPreset(), DataQualityPreset(), TargetDriftPreset()])
report.run(reference_data=image_data_train, current_data=image_data_test)
report.save_html('reports/data_drift_report.html')

data_test = TestSuite(tests=[TestNumberOfMissingValues()])
data_test.run(reference_data=image_data_train, current_data=image_data_test)
data_test.save_html("reports/data_test.html")

c:\Users\duweh\.conda\envs\DL\lib\site-packages\evidently\metrics\data_integrity\dataset_missing_values_metric.py:147: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif missing_value in row.values:
c:\Users\duweh\.conda\envs\DL\lib\site-packages\evidently\metrics\data_integrity\dataset_missing_values_metric.py:147: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

